### tf变量管理

In [2]:
import tensorflow as tf

In [3]:
# tf.get_variable可用于创建或获取变量, 下面二者等价
v = tf.get_variable('v', shape=[1], initializer=tf.constant_initializer(1.0))
v = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

In [9]:
# 已经创建的变量再次创建会报错，增加resuse参数可直接使用已创建的变量
# tf.variable_scope支持嵌套
# 在名字为foo的命名空间内创建名字为v的变量
with tf.variable_scope('foo', reuse=True):
    v1 = tf.get_variable('v', [1], initializer=tf.constant_initializer(1.0))
with tf.variable_scope('foo', reuse=True):
    v2 = tf.get_variable('v', [1], initializer=tf.constant_initializer(1.0))

In [10]:
v1 == v2

True

In [11]:
v1.name

'foo/v:0'

### tf模型持久化
#### 模型保存

In [16]:
# 声明两个变量并计算二者之和
tf.reset_default_graph()
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='v2')
result = v1 + v2
init_op = tf.global_variables_initializer()
# 声明tf.train.saver类用于保存模型
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, 'model/model.ckpt')

#### 模型加载

In [18]:
tf.reset_default_graph() # 重置图，否则会报错
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='v2')
result = v1 + v2
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    # sess.run(init_op) #无需进行变量初始化
    saver.restore(sess, 'model/model.ckpt') # 加载模型，利用模型中的变量计算加法
    print(sess.run(result))

[3.]


In [20]:
# 直接加载持久化的图, 无需定义变量
saver = tf.train.import_meta_graph('model/model.ckpt.meta')
with tf.Session() as sess:
    saver.restore(sess, 'model/model.ckpt') # 加载模型，利用模型中的变量计算加法
    # 通过张量的名称获取变量
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

[3.]


In [29]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='other-v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='other-v2')
saver = tf.train.Saver()

In [33]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
tf.get_variable_scope()

In [34]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
tf.get_variable_scope()

In [32]:
tf.get_variable_scope()

#### 迁移学习

In [36]:
# 将变量取值与图结构保存在一起
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='v2')
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    # 导出当前计算图的GraphDef部分，只需要这一部分就可以完成从输入层到输出层的计算
    graph_def = tf.get_default_graph().as_graph_def()
    # 将图中的变量及其取值转化为常量，同时将图中不必要的节点去掉
    # 最后一个参数['add']给出了需要保存的__计算__节点名称
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    with tf.gfile.GFile('model/combined_model.pb', 'wb') as f:
        f.write(output_graph_def.SerializeToString())

Converted 2 variables to const ops.


In [37]:
# 使用训练好的模型完成迁移学习
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_file = 'model/combined_model.pb'
    with gfile.FastGFile(model_file, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # 将graph_def中保存的图加载到当前的图中。"add:0"为返回的张量名称。保存时给出的是计算节点的名称，"add"
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([3.], dtype=float32)]
